In [1]:
import pandas as pd
import numpy as np

### Merge Dataframe - horizontal

When it comes to translating this to pandas, we can think of the case where we might have these two 
populations as indices in separate DataFrames, maybe with the label of Person Name. When we want to join the
DataFrames together, we have some choices to make. First what if we want a list of all the people regardless
of whether they're staff or student, and all of the information we can get on them? In database terminology,
this is called a full outer join. And in set theory, it's called a union. In the Venn diagram, it represents
everyone in any circle.

Here's an image of what that would look like in the Venn diagram.

![Union](merging2.png)

It's quite possible though that we only want those people who we have maximum information for, those people
who are both staff and students. Maybe being a staff member and a student involves getting a tuition waiver,
and we want to calculate the cost of this. In database terminology, this is called an inner join. Or in set
theory, the intersection. It is represented in the Venn diagram as the overlapping parts of each circle.

Here's what that looks like: ![Intersection](merging3.png)

In [20]:
df1 = pd.DataFrame([[1,1],[2,2],[3,3]],columns=['Name','Role']).set_index("Name")
df2 = pd.DataFrame([[1,4],[2,6],[4,3]],columns=['Name','School']).set_index("Name")
print(df1)
print(df2)

      Role
Name      
1        1
2        2
3        3
      School
Name        
1          4
2          6
4          3


In [7]:
pd.merge(df1,df2,how='inner',left_index=True,right_index=True)

,Role,School
Name,,
1,1,4
2,2,6


In [8]:
pd.merge(df1,df2,how='outer',left_index=True,right_index=True)

,Role,School
Name,,
1,1.0,4.0
2,2.0,6.0
3,3.0,NaN
4,NaN,3.0


In [12]:
#cross is new in version 1.2.0.
pd.merge(df1,df2,how='cross',left_index=True,right_index=True)

ValueError: do not recognize join method cross

In [21]:
pd.merge(df1,df2,how='left',left_index=True,right_index=True)
pd.merge(df1,df2,how='right',left_index=True,right_index=True)

,Role,School
Name,,
1,1.0,4
2,2.0,6
4,NaN,3


In [23]:
df1 = df1.reset_index()
df2 = df2.reset_index()
print(df1)
print(df2)

   Name  Role
0     1     1
1     2     2
2     3     3
   Name  School
0     1       4
1     2       6
2     4       3


In [25]:
pd.merge(df1,df2,how='outer',on='Name')

,Name,Role,School
0,1,1.0,4.0
1,2,2.0,6.0
2,3,3.0,NaN
3,4,NaN,3.0


What happends when Conclict occurs? -> two dataframes have same column

The merge function preserves this information, but appends an _x or _y to help differentiate between which
index went with which column of data. The _x is always the left DataFrame information, and the _y is always
the right DataFrame information.

Here, if we want all the staff information regardless of whether they were students or not. But if they were
students, we would want to get their student details as well.Then we can do a left join and on the column of
Name

In [26]:
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR', 
                          'Location': 'State Street'},
                         {'Name': 'Sally', 'Role': 'Course liasion', 
                          'Location': 'Washington Avenue'},
                         {'Name': 'James', 'Role': 'Grader', 
                          'Location': 'Washington Avenue'}])
student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business', 
                            'Location': '1024 Billiard Avenue'},
                           {'Name': 'Mike', 'School': 'Law', 
                            'Location': 'Fraternity House #22'},
                           {'Name': 'Sally', 'School': 'Engineering', 
                            'Location': '512 Wilson Crescent'}])

pd.merge(staff_df,student_df,on='Name')

,Name,Role,Location_x,School,Location_y
0,Sally,Course liasion,Washington Avenue,Engineering,512 Wilson Crescent
1,James,Grader,Washington Avenue,Business,1024 Billiard Avenue


In [27]:
#Multi-indexing and multiple columns

staff_df = pd.DataFrame([{'First Name': 'Kelly', 'Last Name': 'Desjardins', 
                          'Role': 'Director of HR'},
                         {'First Name': 'Sally', 'Last Name': 'Brooks', 
                          'Role': 'Course liasion'},
                         {'First Name': 'James', 'Last Name': 'Wilde', 
                          'Role': 'Grader'}])
student_df = pd.DataFrame([{'First Name': 'James', 'Last Name': 'Hammond', 
                            'School': 'Business'},
                           {'First Name': 'Mike', 'Last Name': 'Smith', 
                            'School': 'Law'},
                           {'First Name': 'Sally', 'Last Name': 'Brooks', 
                            'School': 'Engineering'}])

pd.merge(staff_df,student_df,on=['First Name', 'Last Name'])

,First Name,Last Name,Role,School
0,Sally,Brooks,Course liasion,Engineering


Question. **How can I merge multiple dataframes?**

### Concatenate Dataframe - Vertical

In [29]:
%%capture
#Magic function should be in first line of the cell
df_2011 = pd.read_csv("datasets/college_scorecard/MERGED2011_12_PP.csv", error_bad_lines=False)
df_2012 = pd.read_csv("datasets/college_scorecard/MERGED2012_13_PP.csv", error_bad_lines=False)
df_2013 = pd.read_csv("datasets/college_scorecard/MERGED2013_14_PP.csv", error_bad_lines=False)

In [34]:
df_2011.head(3)

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,OMAWDP8_NOTFIRSTTIME_POOLED_SUPP,OMENRUP_NOTFIRSTTIME_POOLED_SUPP,OMENRYP_FULLTIME_POOLED_SUPP,OMENRAP_FULLTIME_POOLED_SUPP,OMAWDP8_FULLTIME_POOLED_SUPP,OMENRUP_FULLTIME_POOLED_SUPP,OMENRYP_PARTTIME_POOLED_SUPP,OMENRAP_PARTTIME_POOLED_SUPP,OMAWDP8_PARTTIME_POOLED_SUPP,OMENRUP_PARTTIME_POOLED_SUPP
0,100654.0,100200.0,1002,Alabama A & M University,Normal,AL,35762,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100663.0,105200.0,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100690.0,2503400.0,25034,Amridge University,Montgomery,AL,36117-3553,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df_2011.shape, df_2012.shape, df_2013.shape

((15235, 1977), (7793, 1977), (7804, 1977))

In [38]:
pd.concat([df_2011,df_2012,df_2013]).shape

(30832, 1977)

But wait, now that all the data is
concatenated together, we don't know what observations are from what year anymore! Actually the concat
function has a parameter **that solves such problem with the keys parameter, we can set an extra level of
indices**, we pass in a list of keys that we want to correspond to the dataframes into the keys parameter

In [40]:
pd.concat([df_2011,df_2012,df_2013],keys=[2011,2012,2013])

UNITID      OPEID OPEID6  \
2011 0       100654.0   100200.0   1002   
     1       100663.0   105200.0   1052   
     2       100690.0  2503400.0  25034   
     3       100706.0   105500.0   1055   
     4       100724.0   100500.0   1005   
...               ...        ...    ...   
2013 7799  48285703.0   157107.0   1571   
     7800  48285704.0   157101.0   1571   
     7801  48285705.0   157105.0   1571   
     7802  48285706.0   157100.0   1571   
     7803  48285707.0   157103.0   1571   

                                                  INSTNM            CITY  \
2011 0                          Alabama A & M University          Normal   
     1               University of Alabama at Birmingham      Birmingham   
     2                                Amridge University      Montgomery   
     3               University of Alabama in Huntsville      Huntsville   
     4                          Alabama State University      Montgomery   
...                                                  ...             ...   
2013 7799       Georgia Military College-Columbus Campus        Columbus   
     7800       Georgia Military College-Valdosta Campus        Valdosta   
     7801  Georgia Military College-Warner Robins Campus   Warner Robins   
     7802                Georgia Military College-Online   Milledgeville   
     7803        Georgia Military College-Stone Mountain  Stone Mountain   

          STABBR         ZIP  ACCREDAGENCY INSTURL NPCURL  ...  \
2011 0        AL       35762           NaN     NaN    NaN  ...   
     1        AL  35294-0110           NaN     NaN    NaN  ...   
     2        AL  36117-3553           NaN     NaN    NaN  ...   
     3        AL       35899           NaN     NaN    NaN  ...   
     4        AL  36104-0271           NaN     NaN    NaN  ...   
...          ...         ...           ...     ...    ...  ...   
2013 7799     GA       31909           NaN     NaN    NaN  ...   
     7800     GA       31605           NaN     NaN    NaN  ...   
     7801     GA       31093           NaN     NaN    NaN  ...   
     7802     GA       31061           NaN     NaN    NaN  ...   
     7803     GA       30083           NaN     NaN    NaN  ...   

          OMAWDP8_NOTFIRSTTIME_POOLED_SUPP OMENRUP_NOTFIRSTTIME_POOLED_SUPP  \
2011 0                                 NaN                              NaN   
     1                                 NaN                              NaN   
     2                                 NaN                              NaN   
     3                                 NaN                              NaN   
     4                                 NaN                              NaN   
...                                    ...                              ...   
2013 7799                              NaN                              NaN   
     7800                              NaN                              NaN   
     7801                              NaN                              NaN   
     7802                              NaN                              NaN   
     7803                              NaN                              NaN   

          OMENRYP_FULLTIME_POOLED_SUPP OMENRAP_FULLTIME_POOLED_SUPP  \
2011 0                             NaN                          NaN   
     1                             NaN                          NaN   
     2                             NaN                          NaN   
     3                             NaN                          NaN   
     4                             NaN                          NaN   
...                                ...                          ...   
2013 7799                          NaN                          NaN   
     7800                          NaN                          NaN   
     7801                          NaN                          NaN   
     7802                          NaN                          NaN   
     7803                          NaN                        

Now we have the indices as the year so we know what observations are from what year. You should know that
concatenation also has inner and outer method. If you are concatenating two dataframes that do not have
identical columns, and choose the outer method, some cells will be NaN. If you choose to do inner, then some
observations will be dropped due to NaN values. You can think of this as analogous to the left and right
joins of the merge() function.

### Pandas Idioms

[Idiomatic Python](https://medium.com/the-andela-way/idiomatic-python-coding-the-smart-way-cc560fa5f1d6)


A sort of sub-language within Python, Pandas has its own set of idioms. We've alluded to some of these already, such as using vectorization whenever possible, and not using iterative loops if you don't need to. Several developers and users within the Panda's community have used the term **pandorable** for these idioms. I think it's a great term. So, I wanted to share with you a couple of key features of how you can make your code pandorable.

In [2]:
import pandas as pd
import numpy as np
import timeit

In [6]:
df = pd.read_csv('datasets/census.csv')
df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


#### pandorable 1 : method chaining

In [8]:

# -> Using multiple method
#based on pandas method returns reference of it
(df[df['SUMLEV']==50].set_index(['STNAME','CTYNAME'])
 .rename(columns={'ESTIMATEDBASE2010':'Estimates Base 2010'})).head()

SUMLEV  REGION  DIVISION  STATE  COUNTY  \
STNAME  CTYNAME                                                   
Alabama Autauga County      50       3         6      1       1   
        Baldwin County      50       3         6      1       3   
        Barbour County      50       3         6      1       5   
        Bibb County         50       3         6      1       7   
        Blount County       50       3         6      1       9   

                        CENSUS2010POP  ESTIMATESBASE2010  POPESTIMATE2010  \
STNAME  CTYNAME                                                             
Alabama Autauga County          54571              54571            54660   
        Baldwin County         182265             182265           183193   
        Barbour County          27457              27457            27341   
        Bibb County             22915              22919            22861   
        Blount County           57322              57322            57373   

                        POPESTIMATE2011  POPESTIMATE2012  ...  \
STNAME  CTYNAME                                           ...   
Alabama Autauga County            55253            55175  ...   
        Baldwin County           186659           190396  ...   
        Barbour County            27226            27159  ...   
        Bibb County               22733            22642  ...   
        Blount County             57711            57776  ...   

                        RDOMESTICMIG2011  RDOMESTICMIG2012  RDOMESTICMIG2013  \
STNAME  CTYNAME                                                                
Alabama Autauga County          7.242091         -2.915927         -3.012349   
        Baldwin County         14.832960         17.647293         21.845705   
        Barbour County         -4.728132         -2.500690         -7.056824   
        Bibb County            -5.527043         -5.068871         -6.201001   
        Blount County           1.807375         -1.177622         -1.748766   

                        RDOMESTICMIG2014  RDOMESTICMIG2015  RNETMIG2011  \
STNAME  CTYNAME                                                           
Alabama Autauga County          2.265971         -2.530799     7.606016   
        Baldwin County         19.243287         17.197872    15.844176   
        Barbour County         -3.904217        -10.543299    -4.874741   
        Bibb County            -0.177537          0.177258    -5.088389   
        Blount County          -2.062535         -1.369970     1.859511   

                        RNETMIG2012  RNETMIG2013  RNETMIG2014  RNETMIG2015  
STNAME  CTYNAME                                                             
Alabama Autauga County    -2.626146    -2.722002     2.592270    -2.187333  
        Baldwin County    18.559627    22.727626    20.317142    18.293499  
        Barbour County    -2.758113    -7.167664    -3.978583   -10.543299  
        Bibb County       -4.363636    -5.403729     0.754533     1.107861  
        Blount County     -0.848580    -1.402476    -1.577232    -0.884411  

[5 rows x 98 columns]

In [10]:
#Not pandorable way
df2 = df[df['SUMLEV']==50]
df2 = df2.set_index(['STNAME','CTYNAME'])
df2 = df2.rename(columns={'ESTIMATEDBASE2010':'Estimates Base 2010'})
df2.head()

SUMLEV  REGION  DIVISION  STATE  COUNTY  \
STNAME  CTYNAME                                                   
Alabama Autauga County      50       3         6      1       1   
        Baldwin County      50       3         6      1       3   
        Barbour County      50       3         6      1       5   
        Bibb County         50       3         6      1       7   
        Blount County       50       3         6      1       9   

                        CENSUS2010POP  ESTIMATESBASE2010  POPESTIMATE2010  \
STNAME  CTYNAME                                                             
Alabama Autauga County          54571              54571            54660   
        Baldwin County         182265             182265           183193   
        Barbour County          27457              27457            27341   
        Bibb County             22915              22919            22861   
        Blount County           57322              57322            57373   

                        POPESTIMATE2011  POPESTIMATE2012  ...  \
STNAME  CTYNAME                                           ...   
Alabama Autauga County            55253            55175  ...   
        Baldwin County           186659           190396  ...   
        Barbour County            27226            27159  ...   
        Bibb County               22733            22642  ...   
        Blount County             57711            57776  ...   

                        RDOMESTICMIG2011  RDOMESTICMIG2012  RDOMESTICMIG2013  \
STNAME  CTYNAME                                                                
Alabama Autauga County          7.242091         -2.915927         -3.012349   
        Baldwin County         14.832960         17.647293         21.845705   
        Barbour County         -4.728132         -2.500690         -7.056824   
        Bibb County            -5.527043         -5.068871         -6.201001   
        Blount County           1.807375         -1.177622         -1.748766   

                        RDOMESTICMIG2014  RDOMESTICMIG2015  RNETMIG2011  \
STNAME  CTYNAME                                                           
Alabama Autauga County          2.265971         -2.530799     7.606016   
        Baldwin County         19.243287         17.197872    15.844176   
        Barbour County         -3.904217        -10.543299    -4.874741   
        Bibb County            -0.177537          0.177258    -5.088389   
        Blount County          -2.062535         -1.369970     1.859511   

                        RNETMIG2012  RNETMIG2013  RNETMIG2014  RNETMIG2015  
STNAME  CTYNAME                                                             
Alabama Autauga County    -2.626146    -2.722002     2.592270    -2.187333  
        Baldwin County    18.559627    22.727626    20.317142    18.293499  
        Barbour County    -2.758113    -7.167664    -3.978583   -10.543299  
        Bibb County       -4.363636    -5.403729     0.754533     1.107861  
        Blount County     -0.848580    -1.402476    -1.577232    -0.884411  

[5 rows x 98 columns]

In [13]:
def first_approach():
    global df
    return (df[df['SUMLEV']==50].set_index(['STNAME','CTYNAME'])
             .rename(columns={'ESTIMATEDBASE2010':'Estimates Base 2010'}))

df = pd.read_csv('datasets/census.csv')
timeit.timeit(first_approach, number=100)

0.6252328231930733

In [14]:
def second_approach():
    global df
    df2 = df[df['SUMLEV']==50]
    df2 = df2.set_index(['STNAME','CTYNAME'])
    df2 = df2.rename(columns={'ESTIMATEDBASE2010':'Estimates Base 2010'})
    return df2

df = pd.read_csv('datasets/census.csv')
timeit.timeit(second_approach, number=100)

0.5479692928493023

As you can see, the second approach(line by line approach) is much faster! So, this is a particular example of a classic **time
readability trade off**.

You'll see lots of examples on stack overflow and in documentation of people using method chaining in their
pandas. And so, I think being able to read and understand the syntax is really worth your time. But keep in
mind that **following what appears to be stylistic idioms might have performance issues that you need to
consider as well**.

#### Pandorable 2 : applymap

Now I think applymap is fine, but I
actually rarely use it. Instead, I find myself often wanting to map across all of the rows in a DataFrame.
And pandas has a function that I use heavily there, called apply.

In [1]:
import pandas as pd
df = pd.read_csv('datasets/census.csv')
df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [20]:
def min_max(row):
    data = row[['POPESTIMATE2010',
                'POPESTIMATE2011',
                'POPESTIMATE2012',
                'POPESTIMATE2013',
                'POPESTIMATE2014',
                'POPESTIMATE2015']]
    return pd.Series({'min': np.min(data), 'max': np.max(data)})

In [25]:
result = df.apply(min_max,axis=1).head(10)
result.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [30]:
#Apply function is rarely used with large function definitions. Instead, you
#typically see it used wdith lambdas
rows = ['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']
#df.apply(np.max(df[rows]),axis=1).head() Not using lambda, it causes error
df.apply(lambda x:np.max(x[rows]),axis=1).head()

0    4858979
1      55347
2     203709
3      27341
4      22861
dtype: int64

In [37]:
df.head()
df.loc[:,'SUMLEV':'COUNTY'].apply(np.sum,axis=1).head()

0    50
1    61
2    63
3    65
4    67
dtype: int64

In [38]:
def get_state_region(x):
    northeast = ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 
                 'Rhode Island','Vermont','New York','New Jersey','Pennsylvania']
    midwest = ['Illinois','Indiana','Michigan','Ohio','Wisconsin','Iowa',
               'Kansas','Minnesota','Missouri','Nebraska','North Dakota',
               'South Dakota']
    south = ['Delaware','Florida','Georgia','Maryland','North Carolina',
             'South Carolina','Virginia','District of Columbia','West Virginia',
             'Alabama','Kentucky','Mississippi','Tennessee','Arkansas',
             'Louisiana','Oklahoma','Texas']
    west = ['Arizona','Colorado','Idaho','Montana','Nevada','New Mexico','Utah',
            'Wyoming','Alaska','California','Hawaii','Oregon','Washington']
    
    if x in northeast:
        return "Northeast"
    elif x in midwest:
        return "Midwest"
    elif x in south:
        return "South"
    else:
        return "West"

In [40]:
df['state'] = df['STNAME'].apply(lambda x:get_state_region(x))
df[['state','STNAME']].head()

,state,STNAME
0,South,Alabama
1,South,Alabama
2,South,Alabama
3,South,Alabama
4,South,Alabama


So there are a couple of Pandas idioms. But I think there's many more, and I haven't talked about them here. So here's an unofficial assignment for you. Go look at some of the top ranked questions on pandas on Stack Overflow, and look at how some of the more experienced authors, answer those questions

### groupby

Sometimes we want to select data based on groups and understand aggregated data on a group level. We have seen that even though Pandas allows us to iterate over every row in a dataframe, it is geneally very slow to do so. **Fortunately Pandas has a groupby() function to speed up such task. The idea behind the groupby() function is that it takes some dataframe, splits it into chunks based on some key values, applies computation on those chunks, then combines the results back together into another dataframe.**

In [42]:
import pandas as pd
import numpy as np

In [44]:
df = pd.read_csv('datasets/census.csv')
'CENSUS2010POP' in df.columns

True

In [50]:
%%timeit -n 3

for state in df['STNAME'].unique():
    avg = np.average(df[df['STNAME']==state]['CENSUS2010POP'])
    print('{} : {}'.format(state,avg))
    

Alabama : 140580.4705882353
Alaska : 47348.73333333333
Arizona : 799002.125
Arkansas : 76734.68421052632
California : 1262845.9661016949
Colorado : 154744.4923076923
Connecticut : 794243.7777777778
Delaware : 448967.0
District of Columbia : 601723.0
Florida : 552979.7058823529
Georgia : 121095.6625
Hawaii : 453433.6666666667
Idaho : 69670.3111111111
Illinois : 249138.4854368932
Indiana : 139436.60215053763
Iowa : 60927.1
Kansas : 53832.41509433962
Kentucky : 71725.07438016529
Louisiana : 139488.36923076923
Maine : 156277.76470588235
Maryland : 461884.16
Massachusetts : 873017.2
Michigan : 235324.7619047619
Minnesota : 120543.75
Mississippi : 71501.13253012048
Missouri : 103257.36206896552
Montana : 34716.31578947369
Nebraska : 38858.31914893617
Nevada : 300061.22222222225
New Hampshire : 239358.18181818182
New Jersey : 799263.0909090909
New Mexico : 121128.17647058824
New York : 615177.8412698413
North Carolina : 188821.44554455444
North Dakota : 24910.777777777777
Ohio : 259247.280898

California : 1262845.9661016949
Colorado : 154744.4923076923
Connecticut : 794243.7777777778
Delaware : 448967.0
District of Columbia : 601723.0
Florida : 552979.7058823529
Georgia : 121095.6625
Hawaii : 453433.6666666667
Idaho : 69670.3111111111
Illinois : 249138.4854368932
Indiana : 139436.60215053763
Iowa : 60927.1
Kansas : 53832.41509433962
Kentucky : 71725.07438016529
Louisiana : 139488.36923076923
Maine : 156277.76470588235
Maryland : 461884.16
Massachusetts : 873017.2
Michigan : 235324.7619047619
Minnesota : 120543.75
Mississippi : 71501.13253012048
Missouri : 103257.36206896552
Montana : 34716.31578947369
Nebraska : 38858.31914893617
Nevada : 300061.22222222225
New Hampshire : 239358.18181818182
New Jersey : 799263.0909090909
New Mexico : 121128.17647058824
New York : 615177.8412698413
North Carolina : 188821.44554455444
North Dakota : 24910.777777777777
Ohio : 259247.2808988764
Oklahoma : 96188.48717948717
Oregon : 207085.0810810811
Pennsylvania : 373599.3823529412
Rhode Islan

Vermont : 83432.13333333333
Virginia : 119371.83582089552
Washington : 336227.0
West Virginia : 66178.35714285714
Wisconsin : 155807.83561643836
Wyoming : 46968.833333333336
Alabama : 140580.4705882353
Alaska : 47348.73333333333
Arizona : 799002.125
Arkansas : 76734.68421052632
California : 1262845.9661016949
Colorado : 154744.4923076923
Connecticut : 794243.7777777778
Delaware : 448967.0
District of Columbia : 601723.0
Florida : 552979.7058823529
Georgia : 121095.6625
Hawaii : 453433.6666666667
Idaho : 69670.3111111111
Illinois : 249138.4854368932
Indiana : 139436.60215053763
Iowa : 60927.1
Kansas : 53832.41509433962
Kentucky : 71725.07438016529
Louisiana : 139488.36923076923
Maine : 156277.76470588235
Maryland : 461884.16
Massachusetts : 873017.2
Michigan : 235324.7619047619
Minnesota : 120543.75
Mississippi : 71501.13253012048
Missouri : 103257.36206896552
Montana : 34716.31578947369
Nebraska : 38858.31914893617
Nevada : 300061.22222222225
New Hampshire : 239358.18181818182
New Jers

In [55]:
%%timeit -n 3
#group is key of groupby method, frame is rows in group
for group, frame in df.groupby('STNAME'):
    avg = np.average(frame['CENSUS2010POP'])
    print('{} : {}'.format(group,avg))

Alabama : 140580.4705882353
Alaska : 47348.73333333333
Arizona : 799002.125
Arkansas : 76734.68421052632
California : 1262845.9661016949
Colorado : 154744.4923076923
Connecticut : 794243.7777777778
Delaware : 448967.0
District of Columbia : 601723.0
Florida : 552979.7058823529
Georgia : 121095.6625
Hawaii : 453433.6666666667
Idaho : 69670.3111111111
Illinois : 249138.4854368932
Indiana : 139436.60215053763
Iowa : 60927.1
Kansas : 53832.41509433962
Kentucky : 71725.07438016529
Louisiana : 139488.36923076923
Maine : 156277.76470588235
Maryland : 461884.16
Massachusetts : 873017.2
Michigan : 235324.7619047619
Minnesota : 120543.75
Mississippi : 71501.13253012048
Missouri : 103257.36206896552
Montana : 34716.31578947369
Nebraska : 38858.31914893617
Nevada : 300061.22222222225
New Hampshire : 239358.18181818182
New Jersey : 799263.0909090909
New Mexico : 121128.17647058824
New York : 615177.8412698413
North Carolina : 188821.44554455444
North Dakota : 24910.777777777777
Ohio : 259247.280898

Massachusetts : 873017.2
Michigan : 235324.7619047619
Minnesota : 120543.75
Mississippi : 71501.13253012048
Missouri : 103257.36206896552
Montana : 34716.31578947369
Nebraska : 38858.31914893617
Nevada : 300061.22222222225
New Hampshire : 239358.18181818182
New Jersey : 799263.0909090909
New Mexico : 121128.17647058824
New York : 615177.8412698413
North Carolina : 188821.44554455444
North Dakota : 24910.777777777777
Ohio : 259247.2808988764
Oklahoma : 96188.48717948717
Oregon : 207085.0810810811
Pennsylvania : 373599.3823529412
Rhode Island : 350855.6666666667
South Carolina : 196824.0
South Dakota : 24303.880597014926
Tennessee : 132210.52083333334
Texas : 197220.0862745098
Utah : 184259.0
Vermont : 83432.13333333333
Virginia : 119371.83582089552
Washington : 336227.0
West Virginia : 66178.35714285714
Wisconsin : 155807.83561643836
Wyoming : 46968.833333333336
Alabama : 140580.4705882353
Alaska : 47348.73333333333
Arizona : 799002.125
Arkansas : 76734.68421052632
California : 1262845.

Virginia : 119371.83582089552
Washington : 336227.0
West Virginia : 66178.35714285714
Wisconsin : 155807.83561643836
Wyoming : 46968.833333333336
Alabama : 140580.4705882353
Alaska : 47348.73333333333
Arizona : 799002.125
Arkansas : 76734.68421052632
California : 1262845.9661016949
Colorado : 154744.4923076923
Connecticut : 794243.7777777778
Delaware : 448967.0
District of Columbia : 601723.0
Florida : 552979.7058823529
Georgia : 121095.6625
Hawaii : 453433.6666666667
Idaho : 69670.3111111111
Illinois : 249138.4854368932
Indiana : 139436.60215053763
Iowa : 60927.1
Kansas : 53832.41509433962
Kentucky : 71725.07438016529
Louisiana : 139488.36923076923
Maine : 156277.76470588235
Maryland : 461884.16
Massachusetts : 873017.2
Michigan : 235324.7619047619
Minnesota : 120543.75
Mississippi : 71501.13253012048
Missouri : 103257.36206896552
Montana : 34716.31578947369
Nebraska : 38858.31914893617
Nevada : 300061.22222222225
New Hampshire : 239358.18181818182
New Jersey : 799263.0909090909
New M

Not groupby : 52.5ms vs groupby : 21.3ms 

In [58]:
df.groupby('STNAME')

We could create some function which
returns a number between zero and two based on the first character of the state name. Then we can tell group
by to use this function to split up our data frame. **It's important to note that in order to do this you need
to set the index of the data frame to be the column that you want to group by first.**

In [63]:
def set_batch(item):
    if item[0] < 'M':
        return 0
    elif item[0] < 'Q':
        return 1
    return 2

df2 = df.set_index('STNAME')
for group, frame in df2.groupby(set_batch):
    print("{} : {}".format(len(frame), group))
    
# Notice that this time I didn't pass in a column name to groupby(). Instead, I set the index of the dataframe
# to be STNAME, and if no column identifier is passed groupby() will automatically use the index.

1196 : 0
1154 : 1
843 : 2


In [67]:
df = pd.read_csv('datasets/listings.csv')
df.shape

(3585, 95)

In [70]:
df=df.set_index(["cancellation_policy","review_scores_value"])

for group, frame in df.groupby(level=(0,1)):
    print(group)

('flexible', 2.0)
('flexible', 4.0)
('flexible', 5.0)
('flexible', 6.0)
('flexible', 7.0)
('flexible', 8.0)
('flexible', 9.0)
('flexible', 10.0)
('moderate', 2.0)
('moderate', 4.0)
('moderate', 6.0)
('moderate', 7.0)
('moderate', 8.0)
('moderate', 9.0)
('moderate', 10.0)
('strict', 2.0)
('strict', 3.0)
('strict', 4.0)
('strict', 5.0)
('strict', 6.0)
('strict', 7.0)
('strict', 8.0)
('strict', 9.0)
('strict', 10.0)
('super_strict_30', 6.0)
('super_strict_30', 7.0)
('super_strict_30', 8.0)
('super_strict_30', 9.0)
('super_strict_30', 10.0)


In [73]:
# Check the "review_scores_value" portion of the index. item is in the format of
# (cancellation_policy,review_scores_value]
def group_func(item):
    if item[1]==10.0:
        return (item[0],"10.0")
    else:
        return (item[0],"no 10.0")
for group, frame in df.groupby(group_func):
    print(group)

('flexible', '10.0')
('flexible', 'no 10.0')
('moderate', '10.0')
('moderate', 'no 10.0')
('strict', '10.0')
('strict', 'no 10.0')
('super_strict_30', '10.0')
('super_strict_30', 'no 10.0')


 There are three broad categories of data processing during the apply step
 
 1. Aggregation
 2. Transformation
 3. filtration

#### Aggregation

In [74]:
df = pd.read_csv('datasets/listings.csv')
df.shape

(3585, 95)

In [87]:
df.groupby('cancellation_policy').agg({'review_scores_value':'mean',
                                      'reviews_per_month':('mean','max')})

review_scores_value reviews_per_month       
                                   mean              mean    max
cancellation_policy                                             
flexible                       9.237421          1.829210  12.13
moderate                       9.307398          2.391922  19.15
strict                         9.081441          1.873467  16.30
super_strict_30                8.537313          0.340143   5.11

In [85]:
df.groupby('cancellation_policy').agg(['mean','max',np.nanmean])

id                             scrape_id  \
                             mean       max       nanmean          mean   
cancellation_policy                                                       
flexible             9.947047e+06  14928000  9.947047e+06  2.016091e+13   
moderate             8.091257e+06  14924831  8.091257e+06  2.016091e+13   
strict               8.026556e+06  14933461  8.026556e+06  2.016091e+13   
super_strict_30      2.230111e+06  14798673  2.230111e+06  2.016091e+13   

                                                        host_id            \
                                max       nanmean          mean       max   
cancellation_policy                                                         
flexible             20160906204935  2.016091e+13  3.306772e+07  93854106   
moderate             20160906204935  2.016091e+13  2.285266e+07  93820834   
strict               20160906204935  2.016091e+13  2.205970e+07  93031650   
super_strict_30      20160906204935  2.016091e+13  4.878123e+06  82529160   

                                  host_listings_count  ... license  \
                          nanmean                mean  ... nanmean   
cancellation_policy                                    ...           
flexible             3.306772e+07            5.636637  ...     NaN   
moderate             2.285266e+07           24.971708  ...     NaN   
strict               2.205970e+07           87.530341  ...     NaN   
super_strict_30      4.878123e+06          518.964706  ...     NaN   

                    jurisdiction_names              \
                                  mean max nanmean   
cancellation_policy                                  
flexible                           NaN NaN     NaN   
moderate                           NaN NaN     NaN   
strict                             NaN NaN     NaN   
super_strict_30                    NaN NaN     NaN   

                    calculated_host_listings_count                  \
                                              mean  max    nanmean   
cancellation_policy                                                  
flexible                                  2.294294  136   2.294294   
moderate                                  6.563656   61   6.563656   
strict                                   19.633375  136  19.633375   
super_strict_30                          73.729412   79  73.729412   

                    reviews_per_month                   
                                 mean    max   nanmean  
cancellation_policy                                     
flexible                     1.829210  12.13  1.829210  
moderate                     2.391922  19.15  2.391922  
strict                       1.873467  16.30  1.873467  
super_strict_30              0.340143   5.11  0.340143  

[4 rows x 99 columns]

> :warning: In more recent pandas versions .agg() on a GroupBy object with dictionary is deprecated

[Example.1 for groupby](https://yganalyst.github.io/data_handling/Pd_13/)

[Example.2 for groupby](https://jjangjjong.tistory.com/8)

#### Transformation

Transformation is different from aggregation. Where agg() returns a single value per column, so one row per
group, tranform() returns an object that is the same size as the group. Essentially, **it broadcasts the
function you supply over the grouped dataframe, returning a new dataframe.** This makes combining data later
easy.

In [95]:
cols = ['cancellation_policy','review_scores_value']
df_mean = df[cols].groupby('cancellation_policy').transform(np.nanmean) #return dataframe
df.groupby('cancellation_policy').transform(np.nanmean)['review_scores_value'] #return series

0       9.307398
1       9.307398
2       9.307398
3       9.307398
4       9.237421
          ...   
3580    9.081441
3581    9.081441
3582    9.237421
3583    9.081441
3584    9.237421
Name: review_scores_value, Length: 3585, dtype: float64

In [96]:
df_mean = df_mean.rename({'review_scores_value':'mean_scores'},axis=1)
df = df.merge(df_mean,left_index=True,right_index=True)
df

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,mean_scores
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,f,NaN,NaN,f,moderate,f,f,1,NaN,9.307398
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...",...,f,NaN,NaN,t,moderate,f,f,1,1.30,9.307398
2,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",none,The LOCATION: Roslindale is a safe and diverse...,...,f,NaN,NaN,f,moderate,t,f,1,0.47,9.307398
3,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,none,Roslindale is a lovely little neighborhood loc...,...,f,NaN,NaN,f,moderate,f,f,1,1.00,9.307398
4,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",none,"I love the proximity to downtown, the neighbor...",...,f,NaN,NaN,f,flexible,f,f,1,2.25,9.237421
5,12386020,https://www.airbnb.com/rooms/12386020,20160906204935,2016-09-07,Private Bedroom + Great Coffee,Super comfy bedroom plus your own bathroom in ...,Our sunny condo is located on the second and t...,Super comfy bedroom plus your own bathroom in ...,none,We love our corner of Roslindale! For quiet wa...,...,f,NaN,NaN,f,flexible,f,f,1,1.70,9.237421
6,5706985,https://www.airbnb.com/rooms/5706985,20160906204935,2016-09-07,New Lrg Studio apt 15 min to Boston,It's a 5 minute walk to Rosi Square to catch t...,The whole house was recently redone and it 's ...,It's a 5 minute walk to Rosi Square to catch t...,none,Roslindale is the new hip area of Boston. Clos...,...,f,NaN,NaN,f,strict,f,f,3,4.00,9.081441
7,2843445,https://www.airbnb.com/rooms/2843445,20160906204935,2016-09-07,"""Tranquility"" on ""Top of the Hill""","We can accommodate guests who are gluten-free,...",We provide a bedroom and full shared bath. Ra...,"We can accommodate guests who are gluten-free,...",none,Our neighborhood is residential with friendly ...,...,f,NaN,NaN,f,moderate,t,t,2,2.38,9.307398
8,753446,https://www.airbnb.com/rooms/753446,20160906204935,2016-09-07,6 miles away from downtown Boston!,Nice and cozy apartment about 6 miles away to ...,Nice and cozy apartment about 6 miles away to ...,Nice and cozy apartment about 6 miles away to ...,none,Roslindale is a primarily residential neighbor...,...,f,NaN,NaN,f,moderate,f,f,1,5.36,9.307398
9,849408,https://www.airbnb.com/rooms/849408,20160906204935,2016-09-07,Perfect & Practical Boston Rental,This is a cozy and spacious two bedroom unit w...,Perfect apartment rental for those in town vis...,This is a cozy and spacious two bedroom unit w...,none,"This neighborhood truly has it all. Good, not...",...,f,NaN,NaN,f,strict,f,f,2,1.01,9.081441


In [98]:
np.absolute(df['review_scores_value']-df['mean_scores'])

0            NaN
1       0.307398
2       0.692602
3       0.692602
4       0.762579
          ...   
3580    0.081441
3581         NaN
3582         NaN
3583    2.081441
3584         NaN
Length: 3585, dtype: float64

#### Filtering

The GroupBy object has build in support for filtering groups as well. It's often that you'll want to group
by some feature, then make some transformation to the groups, **then drop certain groups as part of your
cleaning routines. The filter() function takes in a function which it applies to each group dataframe and
returns either a True or a False, depending upon whether that group should be included in the results.**

In [102]:
filter_result = df.groupby('cancellation_policy').filter(lambda x: np.nanmean(x['review_scores_value'])>9.2)
df['cancellation_policy'].unique(),filter_result['cancellation_policy'].unique()
#For lambda x: np.nanmean(x['review_scores_value'])>9.2
#It drops groups that don't match the function

(array(['moderate', 'flexible', 'strict', 'super_strict_30'], dtype=object),
 array(['moderate', 'flexible'], dtype=object))

In [105]:
filter_result2 = df.groupby('cancellation_policy').filter(lambda x:x['review_scores_value']>9.2)

TypeError: filter function returned a Series, but expected a scalar bool

In [106]:
#group is a dataframe whatever we have grouped by, so we can treat group as the
#complete dataframe
def calc_mean_review_scores(group):
    avg=np.nanmean(group["review_scores_value"])
    group["review_scores_mean"]=np.abs(avg-group["review_scores_value"])
    return group

df.groupby('cancellation_policy').apply(calc_mean_review_scores).head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,mean_scores,review_scores_mean
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,NaN,NaN,f,moderate,f,f,1,NaN,9.307398,NaN
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...",...,NaN,NaN,t,moderate,f,f,1,1.30,9.307398,0.307398
2,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",none,The LOCATION: Roslindale is a safe and diverse...,...,NaN,NaN,f,moderate,t,f,1,0.47,9.307398,0.692602
3,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,none,Roslindale is a lovely little neighborhood loc...,...,NaN,NaN,f,moderate,f,f,1,1.00,9.307398,0.692602
4,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",none,"I love the proximity to downtown, the neighbor...",...,NaN,NaN,f,flexible,f,f,1,2.25,9.237421,0.762579


Using apply can be slower than using some of the specialized functions, especially agg(). But, if your
dataframes are not huge, it's a solid general purpose approach

Groupby is a powerful and commonly used tool for data cleaning and data analysis. Once you have grouped the data by some category you have a dataframe of just those values and you can conduct aggregated analsyis on the segments that you are interested. **The groupby() function follows a split-apply-combine approach -** first the data is split into subgroups, then you can apply some transformation, filtering, or aggregation, then the results are combined automatically by pandas for us.

## Scales

Now that we've covered many of the mechanics of pandas, I want to stop and talk for a moment about data types and scales. We've already seen that pandas supports a number of different computational data types such as strings, integers, floating point numbers. What this doesn't capture is what we call the scale of the data.

ex)

A+ - A : 100

A - A- : 80
....

## Ratio scale
 - Units are equally spaced
 - mathematical operations of + - / * are all valid
 - ex) height, weight...
 
## Interval scale
 - Units are equally spaced, but there is no true zero
 - ex) temperature
 
For most of the work that you'll be doing with data mining, the differences between the ratio and interval scales might not be clearly apparent or important to the algorithm that you're to apply, but it's important to have this distinction clear in your mind when applying advanced statistical tests.
 
## Ordinal scale
 - The order of the units is important, but not evenly spaced
 - ex) grade
 
## Nomical scale
 - [Nomical scale](https://www.voxco.com/blog/nominal-scale/)
 - Categories of data, but the categories have no order with respect to one another
 - ex) teams of sports

In [107]:
import pandas as pd
import numpy as np

In [108]:
df=pd.DataFrame(['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D'],
                index=['excellent', 'excellent', 'excellent', 'good', 'good', 'good', 
                       'ok', 'ok', 'ok', 'poor', 'poor'],
               columns=["Grades"])
df

,Grades
excellent,A+
excellent,A
excellent,A-
good,B+
good,B
good,B-
ok,C+
ok,C
ok,C-
poor,D+


In [109]:
df['Grades'].astype('category').head()

excellent    A+
excellent     A
excellent    A-
good         B+
good          B
Name: Grades, dtype: category
Categories (11, object): [A, A+, A-, B, ..., C+, C-, D, D+]

In [120]:
#Our data isn't just categorical, but that it's ordered.
my_categories = pd.CategoricalDtype(categories=['D','D+','C-','C','C+',
                                                'B-','B','B+',
                                                'A-','A','A+'],ordered=True)
my_categories
grades = df['Grades'].astype(my_categories)
df.merge(grades,left_index=True,right_index=True).dtypes
type(grades)

pandas.core.series.Series

Now we see that pandas is not only aware that there are 11 categories, but it is also aware of the order of
those categoreies. So, what can you do with this? Well because there is an ordering this can help with
**comparisons and boolean masking.** For instance, if we have a list of our grades and we compare them to a “C”
we see that the lexicographical comparison returns results we were not intending. 

In [116]:
df[df['Grades']>'C']

,Grades
ok,C+
ok,C-
poor,D+
poor,D


In [119]:
grades[grades>"C"]

excellent    A+
excellent     A
excellent    A-
good         B+
good          B
good         B-
ok           C+
Name: Grades, dtype: category
Categories (11, object): [D < D+ < C- < C ... B+ < A- < A < A+]

Sometimes it is useful to represent categorical values as each being a column with a true or a false as to
whether the category applies. This is especially common in feature extraction, which is a topic in the data
mining course.

 There’s one more common scale-based operation I’d like to talk about, and that’s on converting a scale from
something that is on the interval or ratio scale, like a numeric grade, into one which is categorical. Now,
this might seem a bit counter intuitive to you, since you are losing information about the value. But it’s
commonly done in a couple of places. For instance, if you are visualizing the frequencies of categories,
this can be an extremely useful approach, and histograms are regularly used with converted interval or ratio
data. In addition, if you’re using a machine learning classification approach on data, you need to be using
categorical data, so reducing dimensionality may be useful just to apply a given technique.

In [126]:
df = pd.read_csv('datasets/census.csv')
df = df[df['SUMLEV']==50]
df=df.set_index('STNAME').groupby(level=0)['CENSUS2010POP'].agg(np.average)
df

STNAME
Alabama                  71339.343284
Alaska                   24490.724138
Arizona                 426134.466667
Arkansas                 38878.906667
California              642309.586207
Colorado                 78581.187500
Connecticut             446762.125000
Delaware                299311.333333
District of Columbia    601723.000000
Florida                 280616.567164
Georgia                  60928.635220
Hawaii                  272060.200000
Idaho                    35626.863636
Illinois                125790.509804
Indiana                  70476.108696
Iowa                     30771.262626
Kansas                   27172.552381
Kentucky                 36161.391667
Louisiana                70833.937500
Maine                    83022.562500
Maryland                240564.666667
Massachusetts           467687.785714
Michigan                119080.000000
Minnesota                60964.655172
Mississippi              36186.548780
Missouri                 52077.626087
Monta

In [127]:
pd.cut(df,10)
#pd.cut()
#Makes interval(bin) contains N elements.

STNAME
Alabama                   (11706.087, 75333.413]
Alaska                    (11706.087, 75333.413]
Arizona                 (390320.176, 453317.529]
Arkansas                  (11706.087, 75333.413]
California              (579312.234, 642309.586]
Colorado                 (75333.413, 138330.766]
Connecticut             (390320.176, 453317.529]
Delaware                (264325.471, 327322.823]
District of Columbia    (579312.234, 642309.586]
Florida                 (264325.471, 327322.823]
Georgia                   (11706.087, 75333.413]
Hawaii                  (264325.471, 327322.823]
Idaho                     (11706.087, 75333.413]
Illinois                 (75333.413, 138330.766]
Indiana                   (11706.087, 75333.413]
Iowa                      (11706.087, 75333.413]
Kansas                    (11706.087, 75333.413]
Kentucky                  (11706.087, 75333.413]
Louisiana                 (11706.087, 75333.413]
Maine                    (75333.413, 138330.766]
Maryland     

## Pivot Table

**A pivot table is a way of summarizing data in a DataFrame for a particular purpose.** It makes heavy use of the aggregation function. A pivot table is itself a DataFrame, where the rows represent one variable that you're interested in, the columns another, and the cell's some aggregate value. A pivot table also tends to includes marginal values as well, which are the sums for each column and row. This allows you to be able to see the relationship between two variables at just a glance.

In [2]:
import numpy as np
import pandas as pd

In [136]:
df = pd.read_csv('datasets/cwurData.csv')
df[df['country']=='South Korea']

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
74,75,Seoul National University,South Korea,1,101,34,101,44,101,69,NaN,4,46.74,2012
139,40,Seoul National University,South Korea,1,101,13,101,48,101,76,NaN,4,51.31,2013
223,24,Seoul National University,South Korea,1,355,9,210,38,165,87,107.0,5,66.06,2014
306,107,Yonsei University,South Korea,2,355,61,210,143,330,250,251.0,15,50.95,2014
327,128,Korea Advanced Institute of Science and Techno...,South Korea,3,355,85,210,186,262,363,306.0,11,49.88,2014
329,130,Korea University,South Korea,4,355,58,210,182,428,220,293.0,34,49.84,2014
373,174,Hanyang University,South Korea,5,355,69,210,292,481,406,415.0,28,48.63,2014
385,186,Pohang University of Science and Technology,South Korea,6,355,478,210,279,346,220,295.0,25,48.35,2014
403,204,Sungkyunkwan University,South Korea,7,355,191,210,206,331,187,295.0,54,47.79,2014
513,314,Kyungpook National University,South Korea,8,355,446,210,358,661,283,420.0,68,46.23,2014


In [29]:
df[(df['world_rank']>=1) & (df['world_rank']<100)]
#Method 1
def rank_to_interval(rank):
    if rank in range(101):
        return "Tier 1"
    elif rank in range(101,201):
        return "Tier 2"
    elif rank in range(201,301):
        return "Tier 3"
    else:
        return "Other"
df['rank_level'] = df['world_rank'].apply(rank_to_interval)
#df['rank_level'] = df['world_rank'].apply(lambda x: rank_to_interval(x))
df.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year,rank_level
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012,Tier 1
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012,Tier 1
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012,Tier 1
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012,Tier 1
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012,Tier 1


In [23]:
df = pd.read_csv('datasets/cwurData.csv')
'rank_level' in df.columns

False

In [26]:
#Method 2
#Group by interval and apply
def rank_to_interval(rank):
    if rank in range(101):
        return "Tier 1"
    elif rank in range(101,201):
        return "Tier 2"
    elif rank in range(201,301):
        return "Tier 3"
    else:
        return "Other"
    
grouped = df.groupby(by=rank_to_interval)
for key, dfs in grouped:
    print(key)

Other
Tier 1
Tier 2
Tier 3


A pivot table allows us to pivot out one of these columns a new column headers and compare it against
another column as row indices. Let's say we want to compare rank level versus country of the universities
and we want to compare in terms of overall score

In [36]:
pt = df.pivot_table(values=['score','national_rank'],index='country',columns='rank_level')
pt.head()

national_rank                              score                    \
rank_level         Other Tier 1 Tier 2    Tier 3      Other   Tier 1   Tier 2   
country                                                                         
Argentina       2.285714    NaN    NaN       NaN  44.672857      NaN      NaN   
Australia      17.500000    1.5    4.5  7.000000  44.645750  47.9425  49.2425   
Austria         7.238095    NaN    NaN  1.333333  44.864286      NaN      NaN   
Belgium         8.000000    1.0    2.8  4.666667  45.081000  51.8750  49.0840   
Brazil         10.000000    NaN    1.0       NaN  44.499706      NaN  49.5650   

                       
rank_level     Tier 3  
country                
Argentina         NaN  
Australia   47.285000  
Austria     47.066667  
Belgium     46.746667  
Brazil            NaN

In [38]:
pt = df.pivot_table(values=['score','national_rank'],index='country',columns='rank_level',aggfunc=[np.mean,'max'])
pt.head()

mean                                                       \
           national_rank                              score                     
rank_level         Other Tier 1 Tier 2    Tier 3      Other   Tier 1   Tier 2   
country                                                                         
Argentina       2.285714    NaN    NaN       NaN  44.672857      NaN      NaN   
Australia      17.500000    1.5    4.5  7.000000  44.645750  47.9425  49.2425   
Austria         7.238095    NaN    NaN  1.333333  44.864286      NaN      NaN   
Belgium         8.000000    1.0    2.8  4.666667  45.081000  51.8750  49.0840   
Brazil         10.000000    NaN    1.0       NaN  44.499706      NaN  49.5650   

                                max                                            \
                      national_rank                       score                 
rank_level     Tier 3         Other Tier 1 Tier 2 Tier 3  Other Tier 1 Tier 2   
country                                                                         
Argentina         NaN           4.0    NaN    NaN    NaN  45.66    NaN    NaN   
Australia   47.285000          27.0    2.0    6.0    7.0  45.97  51.61  50.40   
Austria     47.066667          12.0    NaN    NaN    2.0  46.29    NaN    NaN   
Belgium     46.746667          10.0    1.0    4.0    5.0  46.21  52.03  49.73   
Brazil            NaN          18.0    NaN    1.0    NaN  46.08    NaN  49.82   

                   
                   
rank_level Tier 3  
country            
Argentina     NaN  
Australia   47.47  
Austria     47.78  
Belgium     47.14  
Brazil        NaN

In [40]:
#margins
#summarize the values within a given top level column
pt = df.pivot_table(values=['score','national_rank'],index='country',columns='rank_level',
                    aggfunc=[np.mean,'max'],margins=True)
pt.head()

mean                                                \
           national_rank                                         score   
rank_level         Other Tier 1 Tier 2    Tier 3        All      Other   
country                                                                  
Argentina       2.285714    NaN    NaN       NaN   2.285714  44.672857   
Australia      17.500000    1.5    4.5  7.000000  13.137931  44.645750   
Austria         7.238095    NaN    NaN  1.333333   6.500000  44.864286   
Belgium         8.000000    1.0    2.8  4.666667   5.500000  45.081000   
Brazil         10.000000    NaN    1.0       NaN   9.500000  44.499706   

                                                             max         \
                                                   national_rank          
rank_level   Tier 1   Tier 2     Tier 3        All         Other Tier 1   
country                                                                   
Argentina       NaN      NaN        NaN  44.672857           4.0    NaN   
Australia   47.9425  49.2425  47.285000  45.825517          27.0    2.0   
Austria         NaN      NaN  47.066667  45.139583          12.0    NaN   
Belgium     51.8750  49.0840  46.746667  47.011000          10.0    1.0   
Brazil          NaN  49.5650        NaN  44.781111          18.0    NaN   

                                                                 
                              score                              
rank_level Tier 2 Tier 3 All  Other Tier 1 Tier 2 Tier 3    All  
country                                                          
Argentina     NaN    NaN   4  45.66    NaN    NaN    NaN  45.66  
Australia     6.0    7.0  27  45.97  51.61  50.40  47.47  51.61  
Austria       NaN    2.0  12  46.29    NaN    NaN  47.78  47.78  
Belgium       4.0    5.0  10  46.21  52.03  49.73  47.14  52.03  
Brazil        1.0    NaN  18  46.08    NaN  49.82    NaN  49.82

In [42]:
pt.columns,pt.index

(MultiIndex([('mean', 'national_rank',  'Other'),
             ('mean', 'national_rank', 'Tier 1'),
             ('mean', 'national_rank', 'Tier 2'),
             ('mean', 'national_rank', 'Tier 3'),
             ('mean', 'national_rank',    'All'),
             ('mean',         'score',  'Other'),
             ('mean',         'score', 'Tier 1'),
             ('mean',         'score', 'Tier 2'),
             ('mean',         'score', 'Tier 3'),
             ('mean',         'score',    'All'),
             ( 'max', 'national_rank',  'Other'),
             ( 'max', 'national_rank', 'Tier 1'),
             ( 'max', 'national_rank', 'Tier 2'),
             ( 'max', 'national_rank', 'Tier 3'),
             ( 'max', 'national_rank',    'All'),
             ( 'max',         'score',  'Other'),
             ( 'max',         'score', 'Tier 1'),
             ( 'max',         'score', 'Tier 2'),
             ( 'max',         'score', 'Tier 3'),
             ( 'max',         'score',    'All')],

In [44]:
pt['mean','score','Tier 1'] 
pt['mean']['score']['Tier 1'] # Can get same result

country
Argentina                     NaN
Australia               47.942500
Austria                       NaN
Belgium                 51.875000
Brazil                        NaN
Bulgaria                      NaN
Canada                  53.633846
Chile                         NaN
China                   53.592500
Colombia                      NaN
Croatia                       NaN
Cyprus                        NaN
Czech Republic                NaN
Denmark                 49.180000
Egypt                         NaN
Estonia                       NaN
Finland                 44.415000
France                  51.914444
Germany                 49.153636
Greece                        NaN
Hong Kong                     NaN
Hungary                       NaN
Iceland                       NaN
India                         NaN
Iran                          NaN
Ireland                       NaN
Israel                  56.307143
Italy                   48.736667
Japan                   58.812692
Lebano

In [46]:
pt['mean','score','Tier 1'].idxmax()

'United Kingdom'

**stack**

Return a reshaped DataFrame or Series having a multi-level index with one or more new inner-most levels compared to the current DataFrame. The new inner-most levels are created by pivoting the columns of the current dataframe:

**unstack**

Reverse of stack

In [49]:
pt.head()

mean                                                \
           national_rank                                         score   
rank_level         Other Tier 1 Tier 2    Tier 3        All      Other   
country                                                                  
Argentina       2.285714    NaN    NaN       NaN   2.285714  44.672857   
Australia      17.500000    1.5    4.5  7.000000  13.137931  44.645750   
Austria         7.238095    NaN    NaN  1.333333   6.500000  44.864286   
Belgium         8.000000    1.0    2.8  4.666667   5.500000  45.081000   
Brazil         10.000000    NaN    1.0       NaN   9.500000  44.499706   

                                                             max         \
                                                   national_rank          
rank_level   Tier 1   Tier 2     Tier 3        All         Other Tier 1   
country                                                                   
Argentina       NaN      NaN        NaN  44.672857           4.0    NaN   
Australia   47.9425  49.2425  47.285000  45.825517          27.0    2.0   
Austria         NaN      NaN  47.066667  45.139583          12.0    NaN   
Belgium     51.8750  49.0840  46.746667  47.011000          10.0    1.0   
Brazil          NaN  49.5650        NaN  44.781111          18.0    NaN   

                                                                 
                              score                              
rank_level Tier 2 Tier 3 All  Other Tier 1 Tier 2 Tier 3    All  
country                                                          
Argentina     NaN    NaN   4  45.66    NaN    NaN    NaN  45.66  
Australia     6.0    7.0  27  45.97  51.61  50.40  47.47  51.61  
Austria       NaN    2.0  12  46.29    NaN    NaN  47.78  47.78  
Belgium       4.0    5.0  10  46.21  52.03  49.73  47.14  52.03  
Brazil        1.0    NaN  18  46.08    NaN  49.82    NaN  49.82

In [55]:
new_pt = pt.stack()
new_pt.head()
#Lowermost column becomes innermost index
#Some columns whose value is NaN are deleted in result dataframe

max                 mean           
                     national_rank  score national_rank      score
country   rank_level                                              
Argentina All                  4.0  45.66      2.285714  44.672857
          Other                4.0  45.66      2.285714  44.672857
Australia All                 27.0  51.61     13.137931  45.825517
          Other               27.0  45.97     17.500000  44.645750
          Tier 1               2.0  51.61      1.500000  47.942500

In [57]:
new_pt = new_pt.unstack()
new_pt.head()

max                                                  \
           national_rank                             score                 
rank_level           All Other Tier 1 Tier 2 Tier 3    All  Other Tier 1   
country                                                                    
Argentina            4.0   4.0    NaN    NaN    NaN  45.66  45.66    NaN   
Australia           27.0  27.0    2.0    6.0    7.0  51.61  45.97  51.61   
Austria             12.0  12.0    NaN    NaN    2.0  47.78  46.29    NaN   
Belgium             10.0  10.0    1.0    4.0    5.0  52.03  46.21  52.03   
Brazil              18.0  18.0    NaN    1.0    NaN  49.82  46.08    NaN   

                                  mean                                     \
                         national_rank                                      
rank_level Tier 2 Tier 3           All      Other Tier 1 Tier 2    Tier 3   
country                                                                     
Argentina     NaN    NaN      2.285714   2.285714    NaN    NaN       NaN   
Australia   50.40  47.47     13.137931  17.500000    1.5    4.5  7.000000   
Austria       NaN  47.78      6.500000   7.238095    NaN    NaN  1.333333   
Belgium     49.73  47.14      5.500000   8.000000    1.0    2.8  4.666667   
Brazil      49.82    NaN      9.500000  10.000000    NaN    1.0       NaN   

                                                               
                score                                          
rank_level        All      Other   Tier 1   Tier 2     Tier 3  
country                                                        
Argentina   44.672857  44.672857      NaN      NaN        NaN  
Australia   45.825517  44.645750  47.9425  49.2425  47.285000  
Austria     45.139583  44.864286      NaN      NaN  47.066667  
Belgium     47.011000  45.081000  51.8750  49.0840  46.746667  
Brazil      44.781111  44.499706      NaN  49.5650        NaN

In [61]:
new_pt.unstack().head()

                    rank_level  country  
max  national_rank  All         Argentina     4.0
                                Australia    27.0
                                Austria      12.0
                                Belgium      10.0
                                Brazil       18.0
dtype: float64

## Date/Time Func

Pandas has 4 time related classes

1. Timestamp
2. DatetimeIndex
3. Period
4. PeriodIndex

### Timestamp

Single timesampe and associates values with point in time

In [69]:
import time
import datetime as dt
pd.Timestamp(time.time())
pd.Timestamp('9/1/2021 10:05AM')
pd.Timestamp(2019, 12, 20, 0, 0)

Timestamp('2019-12-20 00:00:00')

In [148]:
pd.Timestamp(2021, 9, 19, 0, 0).isoweekday() #Show weekday of given timestamp

7

In [156]:
import pandas as pd
pd.Timestamp('2021/2/20')+pd.offsets.MonthEnd()

Timestamp('2021-02-28 00:00:00')

In [73]:
#Can extract specific daya such as year, minute
pd.Timestamp(2019, 12, 23, 0, 0).year

2019

### Period
Represent single time span, such as a specific day,month or year

In [77]:
pd.Period('1/2021')
pd.Period('9/1/2021 10:05AM')

Period('2021-09-01 10:05', 'T')

In [81]:
#Period objects represent the full timespan that you specify('M')
#Arithmetic on period is easy and intuitive
pd.Period('1/2021')+ 15

Period('2022-04', 'M')

In [82]:
# <Q> : Can I do arithmetic operation on period and period like period1 + period2?
pd.Period('9/1/2021 10:05AM') + pd.Period('1/2022')

TypeError: unsupported operand type(s) for +: 'Period' and 'Period'

### DatetimeIndex and PeriodIndex

In [85]:
#datetimeindex
t1 = pd.Series(list('abc'), [pd.Timestamp('2016-09-01'), pd.Timestamp('2016-09-02'), 
                             pd.Timestamp('2016-09-03')])
t1,type(t1.index)

(2016-09-01    a
 2016-09-02    b
 2016-09-03    c
 dtype: object, pandas.core.indexes.datetimes.DatetimeIndex)

In [87]:
#periodindex
t2 = pd.Series(list('def'), [pd.Period('2016-09'), pd.Period('2016-10'), 
                             pd.Period('2016-11')])
t2,type(t2.index)

(2016-09    d
 2016-10    e
 2016-11    f
 Freq: M, dtype: object, pandas.core.indexes.period.PeriodIndex)

### Convert to datetime

In [92]:
d1 = ['2 June 2013', 'Aug 29, 2014', '2015-06-26', '7/12/16']
df = pd.DataFrame(np.random.randint(10,100,(4,2)),columns=list('ab'),index=d1)
df #It's little horrible to treat because all of them have different time format

,a,b
2 June 2013,57,49
"Aug 29, 2014",46,77
2015-06-26,60,37
7/12/16,69,77


In [93]:
d1 = ['2 June 2013', 'Aug 29, 2014', '2015-06-26', '7/12/16']
d2 = pd.to_datetime(d1)
df = pd.DataFrame(np.random.randint(10,100,(4,2)),columns=list('ab'),index=d2)
df

,a,b
2013-06-02,78,36
2014-08-29,19,50
2015-06-26,10,66
2016-07-12,77,43


In [94]:
#If ambiguous
pd.to_datetime('4.7.12',dayfirst=True)

Timestamp('2012-07-04 00:00:00')

### Timedelta

Interval between two timestamps. It's quite similar to period, but it doesn't have standard time format such as 'D','Y'...

In [97]:
pd.Timestamp('9/3/2022')-pd.Timestamp('2021/08/15')

Timedelta('384 days 00:00:00')

In [102]:
pd.Timestamp('2021/03') + pd.Timedelta('12D 3H')

Timestamp('2021-03-13 03:00:00')

### Offset
Similar to timedelta, but follows specific calendar duration rules

In [104]:
pd.Timestamp('2021/09/15') + 2*pd.offsets.Week()

Timestamp('2021-09-29 00:00:00')

In [106]:
#date_range : If start and end date are not explicitly specified, It considers date as start date
#Like regex, there's sort of a mini language to describe time periods
#ex) 2W-SUN -> biweekly on sunday
dates = pd.date_range('10-01-2022',periods=11,freq='2W-SUN')
dates

DatetimeIndex(['2022-10-02', '2022-10-16', '2022-10-30', '2022-11-13',
               '2022-11-27', '2022-12-11', '2022-12-25', '2023-01-08',
               '2023-01-22', '2023-02-05', '2023-02-19'],
              dtype='datetime64[ns]', freq='2W-SUN')

In [109]:
dates = pd.date_range('10-01-2022',periods=11,freq='QS-JUL')
dates

DatetimeIndex(['2022-10-01', '2023-01-01', '2023-04-01', '2023-07-01',
               '2023-10-01', '2024-01-01', '2024-04-01', '2024-07-01',
               '2024-10-01', '2025-01-01', '2025-04-01'],
              dtype='datetime64[ns]', freq='QS-JUL')

In [141]:
dates = pd.date_range('10-01-2022',periods=11,freq='QS-JUL')
df = pd.DataFrame({'Count 1': 100 + np.random.randint(-5, 10, 11).cumsum(),
                  'Count 2': 120 + np.random.randint(-5, 10, 11)}, index=dates)
df

,Count 1,Count 2
2022-10-01,108,117
2023-01-01,110,120
2023-04-01,115,126
2023-07-01,117,125
2023-10-01,120,121
2024-01-01,129,118
2024-04-01,137,116
2024-07-01,136,128
2024-10-01,143,118
2025-01-01,140,118


In [117]:
np.arange(-5, 10, 1).cumsum()
ex_df = pd.DataFrame(np.arange(-5, 10, 1).cumsum())
ex_df.index.weekday_name

AttributeError: 'RangeIndex' object has no attribute 'weekday_name'

In [119]:
df.index.weekday_name,df.index.month_name

(Index(['Saturday', 'Sunday', 'Saturday', 'Saturday', 'Sunday', 'Monday',
        'Monday', 'Monday', 'Tuesday', 'Wednesday', 'Tuesday'],
       dtype='object'),
 <bound method PandasDelegate._add_delegate_accessors.<locals>._create_delegator_method.<locals>.f of DatetimeIndex(['2022-10-01', '2023-01-01', '2023-04-01', '2023-07-01',
                '2023-10-01', '2024-01-01', '2024-04-01', '2024-07-01',
                '2024-10-01', '2025-01-01', '2025-04-01'],
               dtype='datetime64[ns]', freq='QS-JUL')>)

In [121]:
df.diff() #difference between each date value

,Count 1,Count 2
2022-10-01,NaN,NaN
2023-01-01,3.0,2.0
2023-04-01,4.0,7.0
2023-07-01,8.0,-4.0
2023-10-01,-5.0,6.0
2024-01-01,0.0,-9.0
2024-04-01,-1.0,4.0
2024-07-01,5.0,-6.0
2024-10-01,-1.0,12.0
2025-01-01,-2.0,-4.0


Suppose we want to know what the mean count is for each month in our DataFrame. We can do this using
resample. Converting from a higher frequency from a lower frequency is called downsampling

In [123]:
df.resample('M')

In [128]:
#index의 start date부터 end date 사이가 월 별(마지막날)로 나눠지는 걸 볼 수 있음
df.resample('M').mean()

,Count 1,Count 2
2022-10-31,95.0,115.0
2022-11-30,NaN,NaN
2022-12-31,NaN,NaN
2023-01-31,98.0,117.0
2023-02-28,NaN,NaN
2023-03-31,NaN,NaN
2023-04-30,102.0,124.0
2023-05-31,NaN,NaN
2023-06-30,NaN,NaN
2023-07-31,110.0,120.0


In [129]:
df.resample('Y').mean()

,Count 1,Count 2
2022-12-31,95.00,115.00
2023-12-31,103.75,121.75
2024-12-31,106.50,120.00
2025-12-31,107.00,123.50


### datetime indexing and slicing

In [131]:
df['2023']

,Count 1,Count 2
2023-01-01,98,117
2023-04-01,102,124
2023-07-01,110,120
2023-10-01,105,126


In [133]:
df.loc['2023']

,Count 1,Count 2
2023-01-01,98,117
2023-04-01,102,124
2023-07-01,110,120
2023-10-01,105,126


In [134]:
df['2023':'2025']

,Count 1,Count 2
2023-01-01,98,117
2023-04-01,102,124
2023-07-01,110,120
2023-10-01,105,126
2024-01-01,105,117
2024-04-01,104,121
2024-07-01,109,115
2024-10-01,108,127
2025-01-01,106,123
2025-04-01,108,124


In [142]:
df['2023/10':'2025']

,Count 1,Count 2
2023-10-01,120,121
2024-01-01,129,118
2024-04-01,137,116
2024-07-01,136,128
2024-10-01,143,118
2025-01-01,140,118
2025-04-01,139,117


It's kind of a whirlwind tour we've done here.

정신없는 여행이었다